## Importing required Libraries


In [1]:
using StaticArrays
	using PlutoUI
	import Base: +, *
	using Distributions, Random, Plots, StatsPlots
	plotly()

┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots C:\Users\Hp\.julia\packages\Plots\ldLtW\src\backends.jl:374


Plots.PlotlyBackend()

### Defining the struct - MultiDual

In [3]:
struct MultiDual{N,T} # N-> Size of the vector, T -> DataType of the vector
    val::T
    derivs::SVector{N,T}
end

Basic Functions defined on MultiDual 

In [36]:
function Base.:+(f::MultiDual{N,T}, g::MultiDual{N,T}) where {N,T} # Takes in two Multiduals and performs the addition operation
    return MultiDual{N,T}(f.val + g.val, f.derivs + g.derivs)      # Returns a MultiDual
end
	
function Base.:-(f::MultiDual{N,T}, g::MultiDual{N,T}) where {N,T} # Similar to addition operation
    return MultiDual{N,T}(f.val - g.val, f.derivs - g.derivs)
end

function Base.:*(f::MultiDual{N,T}, g::MultiDual{N,T}) where {N,T} # Takes in two MultiDuals and performs the multiplication operation.
    return MultiDual{N,T}(f.val * g.val, f.val .* g.derivs + g.val .* f.derivs)
end

function Base.:*(f::MultiDual{N,T}, α::Number) where {N,T}  # Performs multipplication operation with a scalar
    return MultiDual{N,T}(f.val * α, f.derivs .* α)
end

Base.:^(f::MultiDual{N, T}, n::Integer) where {N,T} = Base.power_by_squaring(f, n)

Base.:*(α::Number, f::MultiDual{N, T}) where {N, T} = f*α   # We have to define two different methods for the two permutations 
                                                            # of α and MultiDual , promote_rule eliminates this problem .


### Throwing a show method to display a MultiDual

Reference : [I/O](https://docs.julialang.org/en/v1/base/io-network/)

In [8]:
function Base.show(io::IO, ::MIME"text/plain", x::MultiDual{N,T}) where {N,T}   # Writing a similar code as given in the reference link .
	println(io, "MultiDual : ", x.val, " + ", x.derivs,"ϵ") 
end

Defining *a* & *b* as MultiDuals

In [30]:
a=MultiDual(3.0,SVector(1.0,0.0))

MultiDual : 3.0 + [1.0, 0.0]ϵ


In [10]:
b=MultiDual(4.0,SVector(0.0,1.0))

MultiDual : 4.0 + [0.0, 1.0]ϵ


### Defining some more functions on MultiDual
The following functions have been defined :
- Jacobians
- log
- exp
- sin, cos
- abs
- ^(for non integral powers too) 

In [11]:
# Defining the functions over MultiDual
function Base.:log(f::MultiDual{N,T}) where {N,T}
    return MultiDual(log(f.val), inv(f.val) .*f.derivs)
end


function Base.:exp(f::MultiDual{N,T}) where {N,T}
    return MultiDual(exp(f.val), exp(f.val)*f.derivs)
end

function Base.:sin(f::MultiDual{N,T}) where {N,T}
    return MultiDual(sin(f.val),cos(f.val)*f.derivs)
end

function Base.:cos(f::MultiDual{N,T}) where {N,T}
	return MultiDual(cos(f.val),-sin(f.val)*f.derivs)
end	

function Base.:^(f::MultiDual{N,T} , a ::Real) where {N,T}
	return MultiDual((f.val)^a,(a)*((f.val)^(a-1))*f.derivs)
end

function Base.:abs(f::MultiDual{N,T}) where {N,T}
	return MultiDual(abs(f.val),f.derivs)
end
	
function Base.:/(f::MultiDual{N,T} , g::MultiDual{N,T}) where {N,T}
	return MultiDual(f.val/g.val,(f.derivs*g.val-f.val*g.derivs)/g.val*g.val)
end

Applying the defined functions over a & b.

In [12]:
sin(a)

MultiDual : 0.1411200080598672 + [-0.9899924966004454, -0.0]ϵ


In [13]:
cos(a)

MultiDual : -0.9899924966004454 + [-0.1411200080598672, -0.0]ϵ


In [14]:
sin(a)^2+cos(a)^2

MultiDual : 0.9999999999999999 + [0.0, 0.0]ϵ


In [15]:
c=MultiDual(-2.0,SVector(1.0,2.0))
abs(c)

MultiDual : 2.0 + [1.0, 2.0]ϵ


In [16]:
exp(a)

MultiDual : 20.085536923187668 + [20.085536923187668, 0.0]ϵ


In [17]:
log(b)

MultiDual : 1.3862943611198906 + [0.0, 0.25]ϵ


Using ``` convert ``` and ``` promote_rule ``` rule which would eliminate the need of defining two different methods of ```+(::MultiDual, ::Real) ``` and ``` +(::Real, ::MultiDual) ``` . The ``` promote_rule ``` function takes a pair of type objects and returns another type object, such that instances of the argument types will be promoted to the returned type.

[Promotion and Conversion](https://docs.julialang.org/en/v1/manual/conversion-and-promotion/)

In [18]:
import Base:convert,promote_rule
Base.convert(::Type{MultiDual{N, T}}, x::T) where {N, T<:Number} = MultiDual(x, zeros(SVector{N, T}))
Base.promote_rule(::Type{MultiDual{N,T}} , ::Type{T}) where {N,T<:Number}=MultiDual{N,T}

In [22]:
+(promote(a,3)...)   # We can see that the promote rule works well , 
                    # it promotes the real number 3 into MultiDual and then performs the addition operation for two multiduals.

MultiDual : 6 + [1, 0]ϵ


In [23]:
-(promote(4.0,b)...)  # Similar to + opeartion

MultiDual : 0.0 + [0.0, -1.0]ϵ


In [24]:
log(promote(1.0)...)  # 1.0 is converted to MultiDual , and then log operation is applied.

0.0

### Comparison Functions

In [40]:
# Comparing two MultiDuals through their value.
function Base.:>(f::MultiDual{N,T}, g::MultiDual{N,T}) where {N,T}
    if (f.val>g.val)
		return true
	else
		return false
	end
end
	
	
function Base.:<(f::MultiDual{N,T}, g::MultiDual{N,T}) where {N,T}
    if (f.val<g.val)
		return true
	else
		return false
	end
end
	
	
function Base.:(==)(f::MultiDual{N,T}, g::MultiDual{N,T}) where {N,T}
    if (f.val==g.val)
		return true
	else
		return false
	end
end
	
	
# Comparing number & a MultiDual
function Base.:(==)(f::MultiDual{N,T}, g::Number) where {N,T}
    if (f.val==g)
		return true
	else
		return false
	end
end
	
	
function Base.:>(f::MultiDual{N,T}, g::Number) where {N,T}
    if (f.val>g)
		return true
	else
		return false
	end
end
	
	
function Base.:<(f::MultiDual{N,T}, g::Number) where {N,T}
    if (f.val<g)
		return true
	else
		return false
	end
end

In [26]:
a==b

false

In [27]:
b>abs(c)

true

In [31]:
abs(c)>a

false

In [38]:
h(x,y)=(x+y,x^2*y^3)


h (generic function with 1 method)

In [39]:
h(a,b)

(MultiDual{2, Float64}(7.0, [1.0, 1.0]), MultiDual{2, Float64}(576.0, [384.0, 432.0]))